# Access to S³I Directory
This Jupyter Notebook allows to query JSON entries with diverse filters in the S³I Directory. The communication is executed via HTTP REST. Enter the id of your notebook with the corresponding secret as in this script to make this notebook to your private application. In case of problems please feel free to contact the S³I team (s3i@kwh40.de).
Just go to the **Cell** drop-down menu and use the **Run All** button.

In [ ]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64
import getpass
import requests
import collections
from IPython.display import Image
from tools import print_with_timestamp, check_message_encryption, yes, no

## Configure the notebook
In order to send HTTP Request to the S³I Directory, we need to pack an access token into the HTTP Header, which is issued in the S³I IdentityProvider. Enter the id and the secret of your notebook in the following input fields to query an access token. 

In [ ]:
print_with_timestamp("Assign a client to this notebook.")
client_id = input('[S3I]: Please enter your client id:').strip('," ')
client_secret = getpass.getpass('[S3I]: Please enter your client secret:').strip('," ')
print_with_timestamp("Notebook id and secret are set")

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client to access to the S³I Directory. 

In [ ]:
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S³I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=client_id,
                                             client_secret=client_secret,
                                             username=username,
                                             password=password)
"""Get the access token """
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")


After the token is issued and received, you can use it to authenticate yourself in the S³I Directory. As next, we instantiate a S³I Directory object using the S³I Python Lib.

In [ ]:
s3i_dir = s3i.Directory("https://dir.s3i.vswf.dev/api/2/", token=access_token)

## List all JSON entries

In [ ]:
def list_all():
    return s3i_dir.searchAll()["items"]
    

## Search with Type filter

In [ ]:
def list_typed():
    my_type = input("[S³I]: Please enter the fml40 specified type (component, hmi ... ): ")
    return s3i_dir.queryAttributeBased(key="type", value=my_type)


## Searched with name filter

In [ ]:
def list_named():
    my_name = input("[S³I]: Please enter the name of your thing: ")
    return s3i_dir.queryAttributeBased(key="name", value=my_name)

## Searched by owner filter

In [ ]:
def list_owned():
    owner = input("[S³I]: Please enter the owner: ")
    res = requests.get(url="https://config.s3i.vswf.dev/persons/{}".format(owner),
                       headers={"Authorization": "Bearer {}".format(access_token)})
    if res.status_code == 200:
        owner_uuid = res.json()["personIdentity"]["identifier"]
    else:
        print("[S³I]: Can not find the user {} in S³I IdentityProvider!".format(owner))
        return
    return s3i_dir.queryAttributeBased(key="ownedBy", value=owner_uuid)

## Loop for send REST request with filter
In the loop below, you can send a HTTP REST Request with the choosen filter. We actually provide 4 types of filter, If necessary this can be supplemented. In case that the access token expires, you should re-run the whole notebook to obtain the valid access token.  

In [ ]:
while True:
    filter = input("[S³I]: Please enter what kind of filter you want to use: \n-1: No filter \n-2: Type filter \n-3: Name filter \n-4: Owner filter")
    if filter == "1":
        json_res = list_all()
    elif filter == "2":
        json_res = list_typed()
    elif filter == "3":
        json_res = list_named()
    elif filter == "4":
        json_res = list_owned()
    else:
        continue
    if isinstance(json_res, dict):
        print_with_timestamp("searched {} JSON entries: {}".format(len(json_res), json.dumps(json_res, indent=2)))